In [ ]:
# 📦 Import required libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# 📄 Load and preview the datasets (optional)
df_train_preview = pd.read_csv(train_file_path, sep='\t', names=["label", "message"])
df_test_preview = pd.read_csv(test_file_path, sep='\t', names=["label", "message"])
df_train_preview.tail()  # Just to see sample rows

In [ ]:
#Preprocess the data
# Load the datasets again (with column names specified)
train_df = pd.read_csv(train_file_path, sep='\t', names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='\t', names=["label", "message"])

# Convert labels to numeric values (ham = 0, spam = 1)
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Separate features (X) and labels (y)
train_messages = train_df['message'].values
train_labels = train_df['label'].values
test_messages = test_df['message'].values
test_labels = test_df['label'].values

In [ ]:
#Tokenize and pad the text messages

# Define tokenizer settings
vocab_size = 5000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

# Initialize and fit tokenizer on training data
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_messages)

# Convert text messages to sequences of integers
train_sequences = tokenizer.texts_to_sequences(train_messages)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_messages)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [ ]:
# Build and compile the model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # sigmoid for binary classification
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Train the model
model.fit(train_padded, train_labels, epochs=15, validation_data=(test_padded, test_labels), verbose=2)


In [ ]:
#Define function to predict message class
def predict_message(pred_text):
    # Tokenize and pad the input message
    seq = tokenizer.texts_to_sequences([pred_text])
    padded = pad_sequences(seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # Predict using the trained model
    pred = model.predict(padded)[0][0]
    label = "spam" if pred > 0.5 else "ham"

    # Return both the probability and the label
    return [float(pred), label]


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
